In [1]:
!pip install pycryptodome
!pip install cryptography

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.1 MB/s eta 0:00:00


In [ ]:
############ Encryption Code ###########

In [ ]:
import pickle
import os
import zipfile
from Crypto.Cipher import DES3, AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import numpy as np
from PIL import Image
import random, io
import matplotlib.pyplot as plt
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding as sym_padding

# Functions from the first script (Image Encryption)
def shuffle_pixels(image_array, seed):
    np.random.seed(seed)
    flat_array = image_array.flatten()
    shuffled_indices = np.arange(flat_array.shape[0])
    np.random.shuffle(shuffled_indices)
    shuffled_array = flat_array[shuffled_indices].reshape(image_array.shape)
    return shuffled_array, shuffled_indices

def left_circular_shift(array, shift_amount):
    return np.roll(array, -shift_amount)

def image_to_bytes(image):
    with io.BytesIO() as output:
        image.save(output, format="PNG")
        return output.getvalue()

def generate_key_aes():
    return get_random_bytes(16)  # AES key (16 bytes for AES-128)

def generate_key_des3():
    return get_random_bytes(24)  # DES3 key (24 bytes for 3DES)

def generate_xnor_mask(length):
    return get_random_bytes(length)

def xnor_operation(data, mask):
    return bytes(~(d ^ m) & 0xFF for d, m in zip(data, mask))

def layered_encrypt(data, aes_key1, aes_key2, aes_key3, des_key1, des_key2):
    cipher_aes1 = AES.new(aes_key1, AES.MODE_CBC)
    data = cipher_aes1.encrypt(pad(data, AES.block_size))

    cipher_aes2 = AES.new(aes_key2, AES.MODE_CBC)
    data = cipher_aes2.encrypt(pad(data, AES.block_size))

    cipher_aes3 = AES.new(aes_key3, AES.MODE_CBC)
    data = cipher_aes3.encrypt(pad(data, AES.block_size))

    cipher_des1 = DES3.new(des_key1, DES3.MODE_CBC)
    data = cipher_des1.encrypt(pad(data, DES3.block_size))

    cipher_des2 = DES3.new(des_key2, DES3.MODE_CBC)
    data = cipher_des2.encrypt(pad(data, DES3.block_size))

    return data, cipher_aes1.iv, cipher_aes2.iv, cipher_aes3.iv, cipher_des1.iv, cipher_des2.iv

def encrypt_image(image_path):
    try:
        # Load the image
        image = Image.open(Put_your_image_Path).convert("RGB") # Put the Image Path
        image_array = np.array(image)

        # Shuffle pixels
        seed = random.randint(0, 10000)
        shuffled_image_array, shuffled_indices = shuffle_pixels(image_array, seed)

        # Apply left circular shift
        shift_amount = random.randint(0, 100)
        shifted_image_array = left_circular_shift(shuffled_image_array, shift_amount)

        # Convert the image to bytes
        image_bytes = image_to_bytes(Image.fromarray(shifted_image_array))

        # Generate XNOR mask and apply XNOR operation
        xnor_mask = generate_xnor_mask(len(image_bytes))
        xnor_image_bytes = xnor_operation(image_bytes, xnor_mask)

        # Generate AES and DES keys
        aes_key1, aes_key2, aes_key3 = generate_key_aes(), generate_key_aes(), generate_key_aes()
        des_key1, des_key2 = generate_key_des3(), generate_key_des3()

        # Perform layered encryption
        encrypted_data, iv_aes1, iv_aes2, iv_aes3, iv_des1, iv_des2 = layered_encrypt(
            xnor_image_bytes, aes_key1, aes_key2, aes_key3, des_key1, des_key2
        )

        # Save encrypted data and IVs as pickle file
        with open("encrypted_image_data.pkl", "wb") as f:
            pickle.dump((encrypted_data, (iv_aes1, iv_aes2, iv_aes3, iv_des1, iv_des2)), f)

        # Convert encrypted data to an image and save as PNG
        encrypted_image_array = np.frombuffer(encrypted_data[:image_array.size], dtype=np.uint8).reshape(image_array.shape)
        encrypted_image = Image.fromarray(encrypted_image_array)
        encrypted_image.save("encrypted_image.png")

        # Prepare data to save in a text file
        data_to_save = {
            "aes_key1": aes_key1.hex(),
            "aes_key2": aes_key2.hex(),
            "aes_key3": aes_key3.hex(),
            "des_key1": des_key1.hex(),
            "des_key2": des_key2.hex(),
            "seed": seed,
            "shift_amount": shift_amount,
            "xnor_mask": xnor_mask.hex(),
            "shuffled_indices": list(shuffled_indices),
        }

        # Save return values to a text file
        with open("encryption_info.txt", "w") as text_file:
            for key, value in data_to_save.items():
                text_file.write(f"{key}: {value}\n")

        # Zip the information
        zip_filename = "encryption_info.zip"
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            zipf.write("encryption_info.txt", arcname="encryption_info.txt")
            zipf.write("encrypted_image_data.pkl", arcname="encrypted_image_data.pkl")

        # Clean up
        os.remove("encryption_info.txt")

        # Return keys, shuffle seed, shift amount, XNOR mask, and indices
        return aes_key1, aes_key2, aes_key3, des_key1, des_key2, seed, shift_amount, xnor_mask, shuffled_indices

    except Exception as e:
        print(f"Error occurred during encryption: {e}")
        return None


# Function to handle the second part (Encrypting the ZIP using AES and ECC)
def encrypt_files(zip_file_path, recipient_public_key_path):
    try:
        # Load the recipient's ECC public key
        with open(recipient_public_key_path, "rb") as key_file:
            recipient_public_key = serialization.load_pem_public_key(key_file.read(), backend=default_backend())

        # Generate a random AES key (256-bit) and IV
        aes_key = os.urandom(32)
        iv = os.urandom(16)

        # Encrypt the original ZIP file using AES
        with open(zip_file_path, "rb") as zip_file:
            original_data = zip_file.read()

        padder = sym_padding.PKCS7(128).padder()
        padded_data = padder.update(original_data) + padder.finalize()

        cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv), backend=default_backend())
        encrypted_data = cipher.encryptor().update(padded_data)

        encrypted_zip_path = "encrypted_my_files.zip"
        with open(encrypted_zip_path, "wb") as encrypted_file:
            encrypted_file.write(iv + encrypted_data)

        # Generate an ephemeral private key and derive a shared key using ECDH
        ephemeral_private_key = ec.generate_private_key(ec.SECP384R1(), default_backend())
        shared_key = ephemeral_private_key.exchange(ec.ECDH(), recipient_public_key)

        # Derive the AES key encryption key using HKDF
        derived_key = HKDF(
            algorithm=hashes.SHA256(),
            length=32,
            salt=None,
            info=b"encrypted aes key",
            backend=default_backend()
        ).derive(shared_key)

        # Encrypt the AES key using AES-GCM
        aes_key_iv = os.urandom(12)
        aes_key_cipher = Cipher(algorithms.AES(derived_key), modes.GCM(aes_key_iv), backend=default_backend())
        aes_key_encryptor = aes_key_cipher.encryptor()
        encrypted_aes_key = aes_key_encryptor.update(aes_key) + aes_key_encryptor.finalize()

        # Save the encrypted AES key and ephemeral public key
        encrypted_aes_key_path = "encrypted_aes_key.bin"
        with open(encrypted_aes_key_path, "wb") as key_file:
            key_file.write(aes_key_iv + encrypted_aes_key)

        # Return the path to the encrypted AES key and the encrypted zip file
        return encrypted_zip_path, encrypted_aes_key_path

    except Exception as e:
        print(f"Error occurred during file encryption: {e}")
        return None


In [ ]:
########## Decryption Code ##########

In [ ]:
import os
import zipfile
import pickle
from Crypto.Cipher import DES3, AES
from Crypto.Util.Padding import unpad
import numpy as np
from PIL import Image
import io
import re
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import padding as sym_padding


# Helper Functions

# Function to unshuffle pixels based on provided indices
def unshuffle_pixels(shuffled_array, indices):
    flat_array = shuffled_array.flatten()
    unshuffled_array = np.empty_like(flat_array)
    unshuffled_array[indices] = flat_array
    return unshuffled_array.reshape(shuffled_array.shape)

# Function to reverse left circular shift
def reverse_left_circular_shift(array, shift_amount):
    return np.roll(array, shift_amount)

# Function to apply XNOR operation with mask
def xnor_operation(data, mask):
    return bytes(~(d ^ m) & 0xFF for d, m in zip(data, mask))

# Function to perform layered decryption
def layered_decrypt(data, aes_key1, aes_key2, aes_key3, des_key1, des_key2, iv_aes1, iv_aes2, iv_aes3, iv_des1, iv_des2):
    # Reverse DES layers
    cipher_des2 = DES3.new(des_key2, DES3.MODE_CBC, iv_des2)
    data = unpad(cipher_des2.decrypt(data), DES3.block_size)

    cipher_des1 = DES3.new(des_key1, DES3.MODE_CBC, iv_des1)
    data = unpad(cipher_des1.decrypt(data), DES3.block_size)

    # Reverse AES layers
    cipher_aes3 = AES.new(aes_key3, AES.MODE_CBC, iv_aes3)
    data = unpad(cipher_aes3.decrypt(data), AES.block_size)

    cipher_aes2 = AES.new(aes_key2, AES.MODE_CBC, iv_aes2)
    data = unpad(cipher_aes2.decrypt(data), AES.block_size)

    cipher_aes1 = AES.new(aes_key1, AES.MODE_CBC, iv_aes1)
    data = unpad(cipher_aes1.decrypt(data), AES.block_size)

    return data


# Main Decryption Function for Image
def decrypt_image(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract the necessary files from the zip
        zip_ref.extractall("/mnt/data/")
        with open("/mnt/data/encrypted_image_data.pkl", "rb") as f:
            encrypted_data, (iv_aes1, iv_aes2, iv_aes3, iv_des1, iv_des2) = pickle.load(f)

        # Read the encryption info text file
        encryption_params = {}
        with open("/mnt/data/encryption_info.txt", "r") as f:
            for line in f:
                key, value = line.strip().split(": ")
                encryption_params[key] = value

        # Extract parameters from the text file
        aes_key1 = bytes.fromhex(encryption_params["aes_key1"])
        aes_key2 = bytes.fromhex(encryption_params["aes_key2"])
        aes_key3 = bytes.fromhex(encryption_params["aes_key3"])
        des_key1 = bytes.fromhex(encryption_params["des_key1"])
        des_key2 = bytes.fromhex(encryption_params["des_key2"])
        seed = int(encryption_params["seed"])
        shift_amount = int(encryption_params["shift_amount"])
        xnor_mask = bytes.fromhex(encryption_params["xnor_mask"])
        shuffled_indices_str = encryption_params["shuffled_indices"].strip("[]")
        shuffled_indices = [int(re.sub(r'np\.int64\((\d+)\)', r'\1', item)) for item in shuffled_indices_str.split(", ")]

        # Perform layered decryption
        decrypted_data = layered_decrypt(encrypted_data, aes_key1, aes_key2, aes_key3, des_key1, des_key2, iv_aes1, iv_aes2, iv_aes3, iv_des1, iv_des2)

        # Apply XNOR operation to get the original image bytes
        original_image_bytes = xnor_operation(decrypted_data, xnor_mask)

        # Convert bytes back to an image array
        image_array = np.array(Image.open(io.BytesIO(original_image_bytes)))

        # Reverse the circular shift
        shifted_image_array = reverse_left_circular_shift(image_array, shift_amount)

        # Unshuffle pixels
        unshuffled_image_array = unshuffle_pixels(shifted_image_array, shuffled_indices)

        # Convert the unshuffled array back to an image
        decrypted_image = Image.fromarray(unshuffled_image_array)
        # decrypted_image.show()
        # decrypted_image.save(r"D:\CNS_FINAL\decrypted_image.png")  # Save the decrypted image with a filename and extension

        return decrypted_image


# Function to Decrypt AES Key using ECC and HKDF
def decrypt_aes_key(encrypted_aes_key_path, ephemeral_public_key_path, recipient_private_key_path):
    # Load recipient's private key
    with open(recipient_private_key_path, "rb") as key_file:
        recipient_private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )

    # Load the ephemeral public key
    with open(ephemeral_public_key_path, "rb") as eph_key_file:
        ephemeral_public_key = serialization.load_pem_public_key(
            eph_key_file.read(),
            backend=default_backend()
        )

    # Derive a shared key using ECDH
    shared_key = recipient_private_key.exchange(ec.ECDH(), ephemeral_public_key)

    # Derive the AES key encryption key from the shared key using HKDF
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"encrypted aes key",
        backend=default_backend()
    ).derive(shared_key)

    # Load and decrypt the AES key
    with open(encrypted_aes_key_path, "rb") as encrypted_aes_key_file:
        aes_key_iv = encrypted_aes_key_file.read(12)  # First 12 bytes are the AES-GCM IV
        tag = encrypted_aes_key_file.read(16)         # Next 16 bytes are the GCM tag
        encrypted_aes_key = encrypted_aes_key_file.read()  # Remaining bytes are the encrypted AES key

    aes_key_cipher = Cipher(
        algorithms.AES(derived_key),
        modes.GCM(aes_key_iv, tag),
        backend=default_backend()
    )
    aes_key_decryptor = aes_key_cipher.decryptor()
    aes_key = aes_key_decryptor.update(encrypted_aes_key) + aes_key_decryptor.finalize()

    return aes_key


# Main decryption for the entire encrypted ZIP file containing all files
def decrypt_zip(input_zip_path, extracted_dir):
    # Step 1: Extract the contents of final_encrypted_files.zip
    with zipfile.ZipFile(input_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)

    # Paths for extracted files
    encrypted_zip_path = os.path.join(extracted_dir, "encrypted_my_files.zip")
    encrypted_aes_key_path = os.path.join(extracted_dir, "encrypted_aes_key.bin")
    ephemeral_public_key_path = os.path.join(extracted_dir, "ephemeral_public_key.pem")
    recipient_private_key_path = os.path.join(extracted_dir, "ecc_private_key.pem")

    # Step 2: Decrypt the AES key
    aes_key = decrypt_aes_key(encrypted_aes_key_path, ephemeral_public_key_path, recipient_private_key_path)

    # Step 3: Load and decrypt the encrypted ZIP file
    with open(encrypted_zip_path, "rb") as encrypted_file:
        iv = encrypted_file.read(16)  # First 16 bytes are the IV
        encrypted_data = encrypted_file.read()  # Remaining bytes are the encrypted data

    cipher = Cipher(algorithms.AES(aes_key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Unpad the decrypted data
    unpadder = sym_padding.PKCS7(128).unpadder()
    unpadded_data = unpadder.update(decrypted_data) + unpadder.finalize()

    # Step 4: Save the decrypted ZIP file
    decrypted_zip_path = os.path.join(extracted_dir, "decrypted_original.zip")
    with open(decrypted_zip_path, "wb") as decrypted_file:
        decrypted_file.write(unpadded_data)

    print("Decryption complete.")
    print(f"Decrypted original ZIP file saved as '{decrypted_zip_path}'")

    return decrypted_zip_path


# Main execution
if __name__ == "__main__":
    input_zip_path = "final_encrypted_files.zip"    # This is the Encrypted ZIP file Path Which is Comes from The Sender
    extracted_dir = r"D:\CNS_FINAL"     #File location For The Extract the Encrypted Zip File

    # Decrypt the main encrypted ZIP file
    decrypted_zip_path = decrypt_zip(input_zip_path, extracted_dir)

    # Decrypt the image from the decrypted ZIP file
    decrypted_image = decrypt_image(decrypted_zip_path)
    decrypted_image.save(r"D:\CNS_FINAL\decrypted_image.png")   # Save the decrypted image to the File location
    print("Decrypted image saved.")
